In [0]:
%python

# Ensure that the specified directory (dbfs:/user/hive/warehouse/euroteams) is empty. 
dbutils.fs.rm("dbfs:/user/hive/warehouse/euroteams", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse//groupstage_match", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/groupstandings", recurse=True)

Out[3]: False

In [0]:
%sql
CREATE OR REPLACE TABLE EuroTeams(
TeamId INTEGER,
TeamName VARCHAR(255),
GroupName VARCHAR(255)
);

INSERT INTO EuroTeams(TeamId, TeamName, GroupName)
SELECT
    TeamId,
    TeamName,
    CASE
        WHEN TeamId <= 4 THEN 'A'
        WHEN TeamId <= 8 THEN 'B'
        WHEN TeamId <= 12 THEN 'C'
        WHEN TeamId <= 16 THEN 'D'
        WHEN TeamId <= 20 THEN 'E'
        ELSE 'F'
    END AS GroupName
FROM (
    VALUES
    (1, 'Germany'),
    (2, 'Scotland'),
    (3, 'Hungary'),
    (4, 'Switzerland'),
    (5, 'Spain'),
    (6, 'Croatia'),
    (7, 'Italy'),
    (8, 'Albania'),
    (9, 'Slovenia'),
    (10, 'Denmark'),
    (11, 'Serbia'),
    (12, 'England'),
    (13, 'Finland'),
    (14, 'Netherlands'),
    (15, 'Austria'),
    (16, 'France'),
    (17, 'Belgium'),
    (18, 'Slovakia'),
    (19, 'Romania'),
    (20, 'Ukraine'),
    (21, 'Türkiye'),
    (22, 'Greece'),
    (23, 'Portugal'),
    (24, 'Czech')
) AS Teams (TeamId, TeamName);


num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
Select * from Euroteams

TeamId,TeamName,GroupName
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A
5,Spain,B
6,Croatia,B
7,Italy,B
8,Albania,B
9,Slovenia,C
10,Denmark,C


In [0]:
%sql
SELECT *
FROM Euroteams
WHERE GroupName = 'A';

TeamId,TeamName,GroupName
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A


In [0]:
%sql
-- Create the GroupStageMatch table
CREATE OR REPLACE TABLE GroupStage_match(
    MatchId INTEGER,
    Team1 VARCHAR(255),
    Team2 VARCHAR(255),
    Team1Score INTEGER,
    Team2Score INTEGER,
    Result VARCHAR(5),
    MatchWinner VARCHAR(255),
    MatchStatus VARCHAR(20)
);

-- Insert data into the GroupStageMatch table
INSERT INTO GroupStage_match (MatchId, Team1, Team2, Team1Score, Team2Score, Result, MatchWinner, MatchStatus)
SELECT DISTINCT
    ROW_NUMBER() OVER (ORDER BY t1.TeamId) AS MatchId,
    t1.TeamName AS Team1,
    t2.TeamName AS Team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    CONCAT(Team1Score, '-', Team2Score) AS Result,
    CASE
        WHEN Team1Score > Team2Score THEN t1.TeamName
        WHEN Team1Score < Team2Score THEN t2.TeamName
        ELSE 'draw'
    END AS MatchWinner,
    'completed' AS MatchStatus
FROM
    Euroteams t1
JOIN
    Euroteams t2 ON t1.GroupName = t2.GroupName AND t1.TeamId < t2.TeamId
ORDER BY MatchId; 


num_affected_rows,num_inserted_rows
36,36


In [0]:
%sql
Select * from  GroupStage_match

MatchId,Team1,Team2,Team1Score,Team2Score,Result,MatchWinner,MatchStatus
1,Germany,Switzerland,5,4,5-4,Germany,completed
2,Germany,Hungary,1,5,1-5,Hungary,completed
3,Germany,Scotland,1,4,1-4,Scotland,completed
4,Scotland,Switzerland,4,4,4-4,draw,completed
5,Scotland,Hungary,1,1,1-1,draw,completed
6,Hungary,Switzerland,6,3,6-3,Hungary,completed
7,Spain,Albania,6,6,6-6,draw,completed
8,Spain,Italy,2,1,2-1,Spain,completed
9,Spain,Croatia,1,5,1-5,Croatia,completed
10,Croatia,Albania,5,3,5-3,Croatia,completed


Creating the group standings or rankings

In [0]:
%sql
-- Ensure that the table exists with the correct schema
CREATE or REPLACE TABLE GroupStandings (
    TeamId INTEGER,
    TeamName VARCHAR(255),
    GroupName VARCHAR(255),
    MatchesPlayed INTEGER, 
    Wins INTEGER, 
    Draws INTEGER, 
    Losses INTEGER,
    GoalsFor INTEGER, 
    GoalsAgainst INTEGER, 
    GoalDifference INTEGER, 
    Points INTEGER,
    Rankings INTEGER
);

-- Calculate wins, draws, and losses for each team
WITH MatchResults AS (
    SELECT
        Team1 AS TeamName,
        CASE
            WHEN MatchWinner = Team1 THEN 1 
            ELSE 0
        END AS Wins,
        CASE
            WHEN MatchWinner = 'draw' THEN 1
            ELSE 0 
        END AS Draws,
        Team1Score AS GoalsFor,
        Team2Score AS GoalsAgainst
    FROM GroupStage_match
    UNION ALL
    SELECT
        Team2 AS TeamName,
        CASE
            WHEN MatchWinner = Team2 THEN 1
            ELSE 0 
        END AS Wins,
        CASE
            WHEN MatchWinner = 'draw' THEN 1
            ELSE 0 
        END AS Draws,
        Team2Score AS GoalsFor,
        Team1Score AS GoalsAgainst
    FROM GroupStage_match
)
INSERT INTO GroupStandings (TeamId, TeamName, GroupName, MatchesPlayed, Wins, Draws, Losses, GoalsFor, GoalsAgainst, GoalDifference, Points, Rankings)
SELECT
    t.TeamId,
    t.TeamName,
    t.GroupName,
    COUNT(M.Wins) AS MatchesPlayed,
    SUM(M.Wins) AS Wins,
    SUM(M.Draws) AS Draws,
    COUNT(*) - SUM(M.Wins) - SUM(M.Draws) AS Losses,
    SUM(M.GoalsFor) AS GoalsFor,
    SUM(M.GoalsAgainst) AS GoalsAgainst,
    SUM(M.GoalsFor - M.GoalsAgainst) AS GoalDifference,
    (SUM(M.Wins) * 3) + SUM(M.Draws) AS Points,
    0 as Rankings
FROM Euroteams t
LEFT JOIN MatchResults M ON t.TeamName = M.TeamName
GROUP BY t.TeamId, t.TeamName, t.GroupName;


WITH RankedTeams AS (
SELECT
TeamName,
row_number() OVER (PARTITION BY GroupName ORDER BY Points DESC, GoalDifference DESC, GoalsFor DESC, Wins DESC) AS GroupRank
FROM GroupStandings
)
MERGE INTO GroupStandings gs
USING RankedTeams rt
ON gs. TeamName = rt. TeamName
WHEN MATCHED THEN UPDATE SET gs.Rankings = rt.GroupRank;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
24,24,0,0


In [0]:
%sql
-- Select and display the final GroupStandings table with rankings
SELECT * FROM GroupStandings
ORDER BY GroupName, Rankings;


TeamId,TeamName,GroupName,MatchesPlayed,Wins,Draws,Losses,GoalsFor,GoalsAgainst,GoalDifference,Points,Rankings
3,Hungary,A,3,2,1,0,12,5,7,7,1
2,Scotland,A,3,1,2,0,9,6,3,5,2
1,Germany,A,3,1,0,2,7,13,-6,3,3
4,Switzerland,A,3,0,1,2,11,15,-4,1,4
6,Croatia,B,3,3,0,0,14,6,8,9,1
8,Albania,B,3,1,1,1,15,14,1,4,2
5,Spain,B,3,1,1,1,9,12,-3,4,3
7,Italy,B,3,0,0,3,6,12,-6,0,4
9,Slovenia,C,3,3,0,0,15,12,3,9,1
12,England,C,3,2,0,1,12,10,2,6,2



The criteria for determining the best third-placed teams are as follows:

-Points: The teams are ranked based on the number of points earned in the group stage matches (3 points for a win, 1 point for a draw).

-Goal Difference: If two or more teams have the same number of points, their goal difference is considered. Goal difference is calculated by subtracting the total number of goals conceded from the total number of goals scored.

-Goals Scored: If teams still cannot be separated after goal difference, the number of goals scored in all group stage matches is used as the next tiebreaker.

-Fair Play: If teams are still equal, fair play conduct during the tournament is considered. This includes the number of yellow and red cards. The team with fewer disciplinary points advances.

-Drawing of Lots: As a last resort, if teams are still tied after the above criteria, a draw of lots may be used to determine the final rankings.

Positive Goal Difference:

 A team with a positive goal difference (having scored more goals than conceded) is generally considered more favorably than a team with a lower or negative goal difference.
For example, if Team A has a goal difference of +5 (scored 10, conceded 5), and Team B has a goal difference of +2 (scored 8, conceded 6), Team A would be ranked higher in the standings.


Some important points
It is not allowed to use window functions inside WHERE clause.

Round of 16

 Rank all teams within each group based on points, goal difference, goals for, and wins

In [0]:
%sql
CREATE OR REPLACE VIEW Roundof16 AS
WITH CombinedTeams AS (
    (
        SELECT
            Rankings,
            TeamName,
            GroupName
        FROM
           GroupStandings
        WHERE
            Rankings LIKE '1' OR Rankings LIKE '2'
    )
    UNION ALL
    (
        SELECT
            Rankings,
            TeamName,
            GroupName
        FROM
            GroupStandings
        WHERE
            Rankings LIKE '3'
        ORDER BY Points DESC, GoalDifference DESC, GoalsFor DESC, Wins DESC, GoalsAgainst ASC, TeamName
        LIMIT 4
    )
)
SELECT * FROM CombinedTeams
ORDER BY Rankings, GroupName;




In [0]:
%sql
Select * from RoundOf16

Rankings,TeamName,GroupName
1,Hungary,A
1,Croatia,B
1,Slovenia,C
1,Austria,D
1,Ukraine,E
1,Czech,F
2,Scotland,A
2,Albania,B
2,England,C
2,Netherlands,D


In [0]:
%sql
-- %sql
-- DROP VIEW IF EXISTS RoundOf16;

COMBINATION TABLE

In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/combinationforthirdplace", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/matchesfixtures",recurse=True)

Out[5]: True

In [0]:
%sql
DROP TABLE if exists CombinationforThirdPlace

In [0]:
%sql
CREATE or replace  TABLE CombinationforThirdPlace(
  ID INTEGER,
  SELECTEDGROUPS VARCHAR(4),
  TeamB1 CHAR(1),
  TeamC1 CHAR(1),
  TeamE1 CHAR(1),
  TeamF1 CHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO  CombinationForThirdPlace (ID, SELECTEDGROUPS, TeamB1, TeamC1, TeamE1, TeamF1) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql
select * from CombinationforThirdPlace

ID,SELECTEDGROUPS,TeamB1,TeamC1,TeamE1,TeamF1
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


In [0]:
%sql
SELECT
    TeamName,
    CONCAT_WS('', COLLECT_LIST(GroupCombination[0])) AS FinalCombination
FROM (
    SELECT
        TeamName,
        COLLECT_LIST(GroupName) AS GroupCombination
    FROM
        RoundOf16
    WHERE
        Rankings = 3
    GROUP BY
        TeamName
)
GROUP BY
    TeamName;


TeamName,FinalCombination
Spain,B
Serbia,C
Belgium,E
Portugal,F


In [0]:
%sql
CREATE OR REPLACE VIEW MatchedCombinations AS
SELECT
    c.ID,
    c.SELECTEDGROUPS,
    c.TeamB1,
    c.TeamC1,
    c.TeamE1,
    c.TeamF1,
    r.FinalCombination
FROM
    CombinationForThirdPlace c
JOIN (
    SELECT
        REGEXP_REPLACE(CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1))), '\\s', '') AS FinalCombination
    FROM
        RoundOf16
    WHERE
        Rankings = 3
) r ON c.SELECTEDGROUPS = r.FinalCombination;


In [0]:
%sql
SELECT
        REGEXP_REPLACE(CONCAT_WS('', COLLECT_LIST(SUBSTRING(GroupName, 1))), '\\s', '') AS FinalCombination

    FROM
        RoundOf16
    WHERE
        Rankings = 3

FinalCombination
BCEF


In [0]:
%sql
SELECT * FROM MatchedCombinations 

ID,SELECTEDGROUPS,TeamB1,TeamC1,TeamE1,TeamF1,FinalCombination
13,BCEF,F,E,C,B,BCEF


UEFA set out the following schedule for the round of 16:
- Match 1: Winner of Group B vs. 3rd-placed team from Group A/D/E/F

- Match 2: Winner of Group A vs. Runner-up of Group C

- Match 3: Winner of Group F vs. 3rd-placed team from Group A/B/C

- Match 4: Runner-up of Group D vs. Runner-up of Group E

- Match 5: Winner of Group E vs. 3rd-placed team from Group A/B/C/D

- Match 6: Winner of Group D vs. Runner-up of Group F

- Match 7: Winner of Group C vs. 3rd-placed team from Group D/E/F

- Match 8: Runner-up of Group A vs. Runner-up of Group B

In [0]:
%sql
CREATE OR REPLACE TABLE MatchesFixtures (
    MatchID INT,
    Team1 VARCHAR(53),
    Team1Goals INT,
    Team2 VARCHAR(53),
    Team2Goals INT,
    Result VARCHAR(53),
    MatchWinner VARCHAR(53),
    MatchStage VARCHAR(53)
);


Creating a new table matchfixtures for inserting data from the quarterfinals


In [0]:
%sql
-- Insert match fixtures data
INSERT INTO MatchesFixtures (MatchID, Team1, Team1Goals, Team2, Team2Goals, Result, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 
      CASE
        WHEN t1.Rankings = 1 AND t1.GroupName = 'B' THEN 1
        WHEN t1.Rankings = 1 AND t1.GroupName = 'A' THEN 2
        WHEN t1.Rankings = 1 AND t1.GroupName = 'F' THEN 3
        WHEN t1.Rankings = 2 AND t1.GroupName = 'D' THEN 4
        WHEN t1.Rankings = 1 AND t1.GroupName = 'E' THEN 5
        WHEN t1.Rankings = 1 AND t1.GroupName = 'D' THEN 6
        WHEN t1.Rankings = 1 AND t1.GroupName = 'C' THEN 7
        WHEN t1.Rankings = 2 AND t1.GroupName = 'A' THEN 8

      END
    ) AS MatchID,
    t1.TeamName AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    t2.TeamName AS Team2,
    ROUND(RAND() * 6) AS Team2Goals,
    CONCAT(Team1Goals, '-', Team2Goals) AS Result,
    CASE
        WHEN Team1Goals > Team2Goals THEN t1.TeamName
        WHEN Team2Goals > Team1Goals THEN t2.TeamName
        ELSE
          CASE
            WHEN Team1 < Team2 THEN Team1
            ELSE Team2
          END
    END AS MatchWinner,
    'R16' AS MatchStage
FROM RoundOf16 t1
JOIN RoundOf16 t2 ON (
    (t1.Rankings = 1 AND t1.GroupName = 'B' AND t2.Rankings = 3 AND t2.GroupName = (SELECT TeamB1 FROM MatchedCombinations)) OR
    (t1.Rankings = 1 AND t1.GroupName = 'A' AND t2.Rankings = 2 AND t2.GroupName = 'C') OR
    (t1.Rankings = 1 AND t1.GroupName = 'F' AND t2.Rankings = 3 AND t2.GroupName = (SELECT TeamF1 FROM MatchedCombinations)) OR
    (t1.Rankings = 2 AND t1.GroupName = 'D' AND t2.Rankings = 2 AND t2.GroupName = 'E') OR
    (t1.Rankings = 1 AND t1.GroupName = 'E' AND t2.Rankings = 3 AND t2.GroupName = (SELECT TeamE1 FROM MatchedCombinations)) OR
    (t1.Rankings = 1 AND t1.GroupName = 'D' AND t2.Rankings = 2 AND t2.GroupName = 'F') OR
    (t1.Rankings = 1 AND t1.GroupName = 'C' AND t2.Rankings = 3 AND t2.GroupName = (SELECT TeamC1 FROM MatchedCombinations)) OR
    (t1.Rankings = 2 AND t1.GroupName = 'A' AND t2.Rankings = 2 AND t2.GroupName = 'B')
);

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from MatchesFixtures 

MatchID,Team1,Team1Goals,Team2,Team2Goals,Result,MatchWinner,MatchStage
1,Croatia,5,Portugal,5,5.0-5.0,Croatia,R16
2,Hungary,1,England,3,1.0-3.0,England,R16
3,Czech,1,Spain,1,1.0-1.0,Czech,R16
4,Netherlands,1,Romania,3,1.0-3.0,Romania,R16
5,Ukraine,6,Serbia,3,6.0-3.0,Ukraine,R16
6,Austria,4,Türkiye,3,4.0-3.0,Austria,R16
7,Slovenia,5,Belgium,4,5.0-4.0,Slovenia,R16
8,Scotland,2,Albania,4,2.0-4.0,Albania,R16


In [0]:
%sql
INSERT INTO MatchesFixtures (MatchID, Team1, Team1Goals,Team2,Team2Goals, Result, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 8 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    t2.MatchWinner AS Team2,
    ROUND(RAND() * 6) AS Team2Goals,
    CONCAT(t1.Team1Goals, '-', t2.Team2Goals) AS Result,
    'Not played' AS MatchWinner,
    'QuarterFinals' AS MatchStage

FROM MatchesFixtures t1
CROSS JOIN MatchesFixtures t2
ON
  (t1.MatchID = 1 AND t2.MatchId = 2) OR
  (t1.MatchID = 3 AND t2.MatchId = 4) OR
  (t1.MatchID = 5 AND t2.MatchId = 6) OR
  (t1.MatchID = 7 AND t2.MatchId = 8);

UPDATE MatchesFixtures
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'QuarterFinals';

num_affected_rows
4


In [0]:
%sql
select * from MatchesFixtures

MatchID,Team1,Team1Goals,Team2,Team2Goals,Result,MatchWinner,MatchStage
1,Croatia,5,Portugal,5,5.0-5.0,Croatia,R16
2,Hungary,1,England,3,1.0-3.0,England,R16
3,Czech,1,Spain,1,1.0-1.0,Czech,R16
4,Netherlands,1,Romania,3,1.0-3.0,Romania,R16
5,Ukraine,6,Serbia,3,6.0-3.0,Ukraine,R16
6,Austria,4,Türkiye,3,4.0-3.0,Austria,R16
7,Slovenia,5,Belgium,4,5.0-4.0,Slovenia,R16
8,Scotland,2,Albania,4,2.0-4.0,Albania,R16
9,Croatia,5,England,2,5-3,Croatia,QuarterFinals
10,Czech,1,Romania,6,1-3,Romania,QuarterFinals


Semi-Finale


In [0]:
%sql
INSERT INTO MatchesFixtures (MatchID, Team1, Team1Goals,Team2,Team2Goals, Result, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 12 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    t2.MatchWinner AS Team2,
    ROUND(RAND() * 6) AS Team2Goals,
    CONCAT(t1.Team1Goals, '-', t2.Team2Goals) AS Result,
    'Not played' AS MatchWinner,
    'SemiFinals' AS MatchStage

FROM MatchesFixtures t1
CROSS JOIN MatchesFixtures t2
ON
  (t1.MatchID = 9 AND t2.MatchId = 10) OR
  (t1.MatchID = 11 AND t2.MatchId = 12);

UPDATE MatchesFixtures
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'SemiFinals';

num_affected_rows
2


In [0]:
%sql
select * from MatchesFixtures

MatchID,Team1,Team1Goals,Team2,Team2Goals,Result,MatchWinner,MatchStage
1,Croatia,5,Portugal,5,5.0-5.0,Croatia,R16
2,Hungary,1,England,3,1.0-3.0,England,R16
3,Czech,1,Spain,1,1.0-1.0,Czech,R16
4,Netherlands,1,Romania,3,1.0-3.0,Romania,R16
5,Ukraine,6,Serbia,3,6.0-3.0,Ukraine,R16
6,Austria,4,Türkiye,3,4.0-3.0,Austria,R16
7,Slovenia,5,Belgium,4,5.0-4.0,Slovenia,R16
8,Scotland,2,Albania,4,2.0-4.0,Albania,R16
9,Croatia,5,England,2,5-3,Croatia,QuarterFinals
10,Czech,1,Romania,6,1-3,Romania,QuarterFinals


Third place

In [0]:
%sql
INSERT INTO MatchesFixtures (MatchID, Team1, Team1Goals,Team2,Team2Goals, Result, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 14 AS MatchID,
    CASE
      WHEN
        t1.MatchWinner!= t1.Team1 THEN t1.Team1 ELSE t1.Team2
        END AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    CASE
      WHEN
        t2.MatchWinner!= t2.Team2 THEN t2.Team2 ELSE t2.Team1
        END AS Team2,
    ROUND(RAND() * 6) AS Team2Goals,
    CONCAT(t1.Team1Goals, '-', t2.Team2Goals) AS Result,
    'Not Played' AS MatchWinner,
    'ThirdPlace' AS MatchStage
FROM MatchesFixtures t1
CROSS JOIN MatchesFixtures t2
ON
  (t1.MatchID = 13 AND t2.MatchId = 14);

UPDATE MatchesFixtures
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'ThirdPlace';


num_affected_rows
1


In [0]:
%sql
select * from MatchesFixtures

MatchID,Team1,Team1Goals,Team2,Team2Goals,Result,MatchWinner,MatchStage
1,Croatia,5,Portugal,5,5.0-5.0,Croatia,R16
2,Hungary,1,England,3,1.0-3.0,England,R16
3,Czech,1,Spain,1,1.0-1.0,Czech,R16
4,Netherlands,1,Romania,3,1.0-3.0,Romania,R16
5,Ukraine,6,Serbia,3,6.0-3.0,Ukraine,R16
6,Austria,4,Türkiye,3,4.0-3.0,Austria,R16
7,Slovenia,5,Belgium,4,5.0-4.0,Slovenia,R16
8,Scotland,2,Albania,4,2.0-4.0,Albania,R16
9,Croatia,5,England,2,5-3,Croatia,QuarterFinals
10,Czech,1,Romania,6,1-3,Romania,QuarterFinals


Finals

In [0]:
%sql
INSERT INTO MatchesFixtures (MatchID, Team1, Team1Goals,Team2,Team2Goals, Result, MatchWinner, MatchStage)
SELECT
    ROW_NUMBER() OVER (ORDER BY 1) + 15 AS MatchID,
    t1.MatchWinner AS Team1,
    ROUND(RAND() * 6) AS Team1Goals,
    t2.MatchWinner AS Team2,
    ROUND(RAND() * 6) AS Team2Goals,
    CONCAT(t1.Team1Goals, '-', t2.Team2Goals) AS Result,
    'Not played' AS MatchWinner,
    'Finals' AS MatchStage

FROM MatchesFixtures t1
CROSS JOIN MatchesFixtures t2
ON
  (t1.MatchID = 13 AND t2.MatchId = 14);

UPDATE MatchesFixtures
SET MatchWinner = 
    CASE
        WHEN Team1Goals > Team2Goals THEN Team1
        WHEN Team1Goals < Team2Goals THEN Team2
        ELSE
            CASE
                WHEN Team1 < Team2 THEN Team1
                ELSE Team2
            END
    END
WHERE MatchStage = 'Finals';

num_affected_rows
1


In [0]:
%sql
select * from MatchesFixtures

MatchID,Team1,Team1Goals,Team2,Team2Goals,Result,MatchWinner,MatchStage
1,Croatia,5,Portugal,5,5.0-5.0,Croatia,R16
2,Hungary,1,England,3,1.0-3.0,England,R16
3,Czech,1,Spain,1,1.0-1.0,Czech,R16
4,Netherlands,1,Romania,3,1.0-3.0,Romania,R16
5,Ukraine,6,Serbia,3,6.0-3.0,Ukraine,R16
6,Austria,4,Türkiye,3,4.0-3.0,Austria,R16
7,Slovenia,5,Belgium,4,5.0-4.0,Slovenia,R16
8,Scotland,2,Albania,4,2.0-4.0,Albania,R16
9,Croatia,5,England,2,5-3,Croatia,QuarterFinals
10,Czech,1,Romania,6,1-3,Romania,QuarterFinals
